From Tensor SkFlow: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/iris_custom_model.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import datasets, metrics, cross_validation
from tensorflow.contrib import learn

import chainer.functions as F
import chainer.links as L
from chainer import optimizers, Chain
from commonml.skchainer import ChainerEstimator, SoftmaxCrossEntropyClassifier

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20

In [ ]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = cross_validation.train_test_split(iris.data, iris.target,
    test_size=0.2, random_state=42)

In [ ]:
class Model(Chain):

    def __init__(self, in_size):
        super(Model, self).__init__(l1=L.Linear(in_size, 10),
                                    l2=L.Linear(10, 20),
                                    l3=L.Linear(20, 10),
                                    l4=L.Linear(10, 3),
                                   )

    def __call__(self, x, train=True):
        h1 = F.relu(self.l1(x))
        h2 = F.dropout(F.relu(self.l2(h1)), ratio=.1, train=train)
        h3 = F.dropout(F.relu(self.l3(h2)), ratio=.1, train=train)
        h4 = self.l4(h3)
        return h4

classifier = ChainerEstimator(model=SoftmaxCrossEntropyClassifier(Model(X_train.shape[1])),
                              optimizer=optimizers.AdaGrad(lr=0.1),
                              batch_size=100,
                              device=0,
                              stop_trigger=(100, 'epoch'))

classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print('Accuracy: {0:f}'.format(score))